# RD coördinaten omzetten naar WGS84 coördinaten

In [1]:
import sys
print(f'python {sys.version_info.major}.{sys.version_info.minor}.{sys.version_info.micro}')

import pyproj
print(f'pyproj {pyproj.__version__}')
      
import folium
print(f'folium {folium.__version__}')

python 3.6.6
pyproj 1.9.5.1
folium 0.7.0


## 1. PyProj

In [2]:
# RD coördinaten Onze Lieve Vrouwetoren
x, y = 155000, 463000

In [3]:
# Rijkdriehoek (RD)¶
rd = pyproj.Proj(init='epsg:28992') # http://epsg.io/28992

In [4]:
# World Geodetic System 1984, used in GPS (WGS84)¶
wgs84 = pyproj.Proj(init='epsg:4326') # http://epsg.io/4326

In [5]:
# Transformeer projectie
wgs84_x, wgs84_y = pyproj.transform(rd, wgs84, x, y)
print(wgs84_x, wgs84_y)

5.3872035087259045 52.15517230115187


In [6]:
# Toon kaart
m = folium.Map(location = (wgs84_y, wgs84_x), zoom_start = 20)
folium.Marker(location = (wgs84_y, wgs84_x),
              popup = f'<b>Onze Lieve Vrouwetoren</b><br>x = {wgs84_x}<br>y = {wgs84_y}').add_to(m)
display(m)

## 2. RDWGSConverter

In [7]:
class RDWGSConverter:
    X0 = 155000
    Y0 = 463000
    phi0 = 52.15517440
    lam0 = 5.38720621

    def fromRdToWgs(self, coords):

        Kp = [0, 2, 0, 2, 0, 2, 1, 4, 2, 4, 1]
        Kq = [1, 0, 2, 1, 3, 2, 0, 0, 3, 1, 1]
        Kpq = [3235.65389, -32.58297, -0.24750, -0.84978, -0.06550, -0.01709, -0.00738, 0.00530, -0.00039, 0.00033, -0.00012]

        Lp = [1, 1, 1, 3, 1, 3, 0, 3, 1, 0, 2, 5]
        Lq = [0, 1, 2, 0, 3, 1, 1, 2, 4, 2, 0, 0]
        Lpq = [5260.52916, 105.94684, 2.45656, -0.81885, 0.05594, -0.05607, 0.01199, -0.00256, 0.00128, 0.00022, -0.00022, 0.00026]

        dX = 1E-5 * (coords[0] - self.X0)
        dY = 1E-5 * (coords[1] - self.Y0)

        phi = 0
        lam = 0

        for k in range(len(Kpq)):
            phi = phi + (Kpq[k] * dX ** Kp[k] * dY ** Kq[k])
        phi = self.phi0 + phi / 3600

        for l in range(len(Lpq)):
            lam = lam + (Lpq[l] * dX ** Lp[l] * dY ** Lq[l])
        lam = self.lam0 + lam / 3600

        return [phi, lam]

    def fromWgsToRd(self, coords):

        Rp = [0, 1, 2, 0, 1, 3, 1, 0, 2]
        Rq = [1, 1, 1, 3, 0, 1, 3, 2, 3]
        Rpq = [190094.945, -11832.228, -114.221, -32.391, -0.705, -2.340, -0.608, -0.008, 0.148]

        Sp = [1, 0, 2, 1, 3, 0, 2, 1, 0, 1]
        Sq = [0, 2, 0, 2, 0, 1, 2, 1, 4, 4]
        Spq = [309056.544, 3638.893, 73.077, -157.984, 59.788, 0.433, -6.439, -0.032, 0.092, -0.054]

        dPhi = 0.36 * (coords[0] - self.phi0)
        dLam = 0.36 * (coords[1] - self.lam0)

        X = 0
        Y = 0

        for r in range(len(Rpq)):
            X = X + (Rpq[r] * dPhi ** Rp[r] * dLam ** Rq[r])
        X = self.X0 + X

        for s in range(len(Spq)):
            Y = Y + (Spq[s] * dPhi ** Sp[s] * dLam ** Sq[s])
        Y = self.Y0 + Y

        return [X, Y]

In [8]:
# RD coördinaten Onze Lieve Vrouwetoren
x, y = 155000, 463000

In [9]:
# Converter
converter = RDWGSConverter()

In [10]:
# Converteer
wgsCoords = converter.fromRdToWgs([x, y]) # van RD naar WGS84
rdCoords = converter.fromWgsToRd(wgsCoords) # van WGS84 naar RD
print(wgsCoords, rdCoords )

[52.1551744, 5.38720621] [155000.0, 463000.0]


In [11]:
# Toon kaart
m = folium.Map(location = wgsCoords, zoom_start = 20)
folium.Marker(location = wgsCoords,
              popup = f'<b>Onze Lieve Vrouwetoren</b><br>x = {wgsCoords[1]}<br>y = {wgsCoords[0]}').add_to(m)
display(m)

## 3. Vergelijking

In [12]:
# RD coördinaten
coords = {'Martinitoren': (233883.131, 582065.167),
          'Onze Lieve Vrouwetoren': (155000, 463000),
          'Westertoren': (120700.723, 487525.501)}

In [13]:
# PyProj
for toren in coords.keys():
    x, y = pyproj.transform(rd, wgs84, coords[toren][0], coords[toren][0])
    print('{:24}x = {:7.5f}  y = {:8.5f}'.format(toren, x, y))

Martinitoren            x = 6.48928  y = 50.09043
Onze Lieve Vrouwetoren  x = 5.38719  y = 49.38677
Westertoren             x = 4.91801  y = 49.07756


In [14]:
# RDWGSConverter
for toren in coords.keys():
    wgsCoords = converter.fromRdToWgs([coords[toren][0], coords[toren][0]])
    print('{:24}x = {:7.5f}  y = {:8.5f}'.format(toren, wgsCoords[1], wgsCoords[0]))

Martinitoren            x = 6.48929  y = 50.09044
Onze Lieve Vrouwetoren  x = 5.38720  y = 49.38677
Westertoren             x = 4.91801  y = 49.07756


##### bronnen

* RDWGSConverter: [https://thomasv.nl/2014/03/rd-naar-gps](https://thomasv.nl/2014/03/rd-naar-gps).  
Typo in de class `RDWGSConverter` gecorrigeerd: Kpq -0.0039 moet zijn -0.00039 (-3.9E-4 in class `Converter` conform transformatieformules).
* Transformatieformules: [https://media.thomasv.nl/2015/07/Transformatieformules.pdf](https://media.thomasv.nl/2015/07/Transformatieformules.pdf)